In [26]:
%load_ext autoreload
%autoreload 2


In [58]:
from imports import *

from datasets.bdd_oia import BDD_OIA

from decision_generator_model import DecisionGenerator_v3,DecisionGenerator


device = torch.device("cuda:1")
batch_size = 10
torch.cuda.empty_cache()


## Data loader
image_dir = './data/bdd_oia/lastframe/data/'
label_dir = './data/bdd_oia/lastframe/labels/'


# bdd_oia_dataset = BDD_OIA(image_dir,label_dir+'train_25k_images_actions.json',
#                              label_dir+'train_25k_images_reasons.json')

# training_loader = DataLoader(bdd_oia_dataset,
#                             shuffle=True,
#                             batch_size=batch_size,
#                             num_workers=0,
#                             drop_last=True,
#                             collate_fn=utils.collate_fn)


val_bdd_oia_dataset = BDD_OIA(image_dir,label_dir+'val_25k_images_actions.json',
                             label_dir+'val_25k_images_reasons.json')

val_loader = DataLoader(val_bdd_oia_dataset,shuffle=False,batch_size=batch_size,collate_fn=utils.collate_fn)

batch_size = 10

def get_model(num_classes):
    # model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True,
    #                                                             image_mean=image_mean,
    #                                                             image_std=image_std)

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features,num_classes)
    return model



fastercnn = get_model(10)
checkpoint = torch.load('saved_models/bdd100k_24.pth')
fastercnn.load_state_dict(checkpoint['model'])

decision_generator = DecisionGenerator(fastercnn,device,batch_size)
decision_generator = decision_generator.to(device)

checkpoint = torch.load("/home/ai/Desktop/Jiqian work/work4/saved_models/bdd_oia_sgd269.pth")
decision_generator.load_state_dict(checkpoint["model"])

decision_generator = decision_generator.to(device)
decision_generator.eval()



2259 2259 2259
number of samples in dataset:2259


DecisionGenerator(
  (rcnn): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): FrozenBatchNorm2d

## 2

In [59]:
from sklearn.metrics import f1_score
def List2Arr(List):
    Arr1 = np.array(List[:-1]).reshape(-1, List[0].shape[1])
    Arr2 = np.array(List[-1]).reshape(-1, List[0].shape[1])

    return np.vstack((Arr1, Arr2))

def ComputeClsAcc(target, pred):
    """
    target - target array, (N, cls)
    pred - prediction array (N, cls)
    """
    f1_macro = []
    f1_micro = []
    for cls in range(target.shape[1]):
        target_ = target[:, cls]
        pred_ = pred[:, cls]
        f1_macro.append(f1_score(target_, pred_, average='macro'))
        f1_micro.append(f1_score(target_, pred_, average='micro'))
    f1_macro = np.array(f1_macro)
    f1_micro = np.array(f1_micro)

    return f1_macro, f1_micro

In [60]:
count = val_loader.__len__()
for i,databatch in enumerate(val_loader):
    print('Finished: {} / {}'.format(i, count))
    print('Finished: %.2f%%' % (i /count * 100))

    images, targets = databatch
    
    break
#     images = list(image.to(device) for image in images)
#     targets = [{k: v.to(device) for k, v in t.items()} for t in targets]   

#     actions, reasons = decision_generator(images)

#     actions.data.cpu().numpy()

Finished: 0 / 226
Finished: 0.00%


In [61]:
images_cuda = list(image.to(device) for image in images)

pred = decision_generator(images_cuda)
pred_action = pred['action'].detach().cpu().numpy()
pred_reason = pred['reasons'].detach().cpu().numpy()

In [62]:
pred_action
# pred_reason>0.5

array([[0.02105572, 0.9771692 , 0.04507454, 0.08204881],
       [0.6420803 , 0.3368849 , 0.4013624 , 0.7208964 ],
       [0.00152936, 0.9986267 , 0.09504992, 0.09827093],
       [0.91608244, 0.11679319, 0.28588438, 0.24264814],
       [0.48801255, 0.49125808, 0.40134308, 0.63624513],
       [0.8248857 , 0.22827734, 0.5718797 , 0.6507185 ],
       [0.03020869, 0.983763  , 0.02891728, 0.03394839],
       [0.6957603 , 0.33953938, 0.5678243 , 0.6645584 ],
       [0.95622814, 0.04952107, 0.5731888 , 0.5244346 ],
       [0.81095105, 0.20496988, 0.47319394, 0.45550692]], dtype=float32)

In [63]:
pred_reason 

array([[1.18962079e-02, 6.57967804e-03, 1.40614519e-02, 1.81847543e-01,
        4.07783180e-01, 2.81956885e-02, 3.68359452e-03, 9.58213449e-01,
        3.40869930e-03, 2.73512164e-03, 5.74760232e-03, 3.33934836e-03,
        2.58290931e-03, 3.11982743e-02, 9.49960761e-03, 9.47630927e-02,
        5.33132404e-02, 9.68521610e-02, 1.61667749e-01, 7.41013512e-02,
        4.85514626e-02],
       [1.21031120e-01, 4.30688381e-01, 1.32025510e-01, 9.79197770e-03,
        3.88129652e-02, 2.26819818e-03, 3.85182514e-03, 3.22101235e-01,
        8.80975090e-03, 4.89834056e-04, 1.45984283e-02, 3.01777013e-03,
        4.92203457e-04, 1.60281472e-02, 3.40182520e-03, 1.04700014e-01,
        1.37599394e-01, 1.50542289e-01, 4.78945598e-02, 1.14171155e-01,
        8.64966288e-02],
       [1.27193739e-03, 3.14118864e-04, 8.15395208e-04, 7.62618065e-01,
        9.07756016e-02, 8.84258561e-03, 7.95045169e-04, 9.95096862e-01,
        8.35836574e-04, 2.47755786e-04, 1.49661486e-04, 4.81499592e-05,
        1.4002

In [64]:
pred_reason

array([[1.18962079e-02, 6.57967804e-03, 1.40614519e-02, 1.81847543e-01,
        4.07783180e-01, 2.81956885e-02, 3.68359452e-03, 9.58213449e-01,
        3.40869930e-03, 2.73512164e-03, 5.74760232e-03, 3.33934836e-03,
        2.58290931e-03, 3.11982743e-02, 9.49960761e-03, 9.47630927e-02,
        5.33132404e-02, 9.68521610e-02, 1.61667749e-01, 7.41013512e-02,
        4.85514626e-02],
       [1.21031120e-01, 4.30688381e-01, 1.32025510e-01, 9.79197770e-03,
        3.88129652e-02, 2.26819818e-03, 3.85182514e-03, 3.22101235e-01,
        8.80975090e-03, 4.89834056e-04, 1.45984283e-02, 3.01777013e-03,
        4.92203457e-04, 1.60281472e-02, 3.40182520e-03, 1.04700014e-01,
        1.37599394e-01, 1.50542289e-01, 4.78945598e-02, 1.14171155e-01,
        8.64966288e-02],
       [1.27193739e-03, 3.14118864e-04, 8.15395208e-04, 7.62618065e-01,
        9.07756016e-02, 8.84258561e-03, 7.95045169e-04, 9.95096862e-01,
        8.35836574e-04, 2.47755786e-04, 1.49661486e-04, 4.81499592e-05,
        1.4002

In [65]:
target_action = np.stack([t['action'].numpy() for t in targets])
target_reason = np.stack([t['reason'].numpy() for t in targets])

In [66]:
f1_action = f1_score(pred_action>0.5, target_action, average='samples')
f1_reason = f1_score(pred_reason>0.5, target_reason, average='samples')


In [67]:
f1_action,f1_reason

(0.8933333333333333, 0.5090476190476191)

In [68]:
np.hstack([target_action,1*(pred_action>0.5)])

array([[0., 1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0., 1., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 1., 1., 1., 0., 1., 1.],
       [1., 0., 0., 1., 1., 0., 1., 1.],
       [1., 0., 0., 1., 1., 0., 0., 0.]])

In [57]:
model_name = "bdd_oia_sgd249"


act = np.load("./saved_predictions/pred_action_%s.npy"%model_name)
tar_act = np.load("./saved_predictions/trgt_action_%s.npy"%model_name)
rsn = np.load("./saved_predictions/pred_reason_%s.npy"%model_name)
tar_rsn = np.load("./saved_predictions/trgt_reason_%s.npy"%model_name)
f1_score(act,tar_act,average='samples'),f1_score(rsn,tar_rsn,average='samples')

FileNotFoundError: [Errno 2] No such file or directory: './saved_predictions/pred_action_bdd_oia_sgd249.npy'

0.6654693395729252